## Contributors
### Jay Upadhyay
### 
###
###
###

# Libraries

In [1]:
# from tensorflow import keras
# from keras import utils
# from keras.models import Sequential
# from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import numpy as np
import h5py
import pandas as pd
import seaborn as sns

In [2]:
df_orig=pd.read_csv("adult.csv")
df_orig.dropna(inplace=True)
df_orig

,Age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week\t,native-country\t,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
# Changing a column "class" to be 0:<=50K and 1:>50K
class_list = list(sorted(set(df_orig["class"])))

for i,x in enumerate(class_list):
    print(i,":", x)
    
df_orig['class'].replace(class_list,
                        range(len(class_list)), inplace=True)

0 :  <=50K
1 :  >50K


In [4]:
# Changing a column "class" to be 0:<=50K and 1:>50K
country_list = list(sorted(set(df_orig["native-country\t"])))

for i,x in enumerate(country_list):
    print(i,":", x)

df_orig['native-country\t'].replace(country_list,
                        range(len(country_list)), inplace=True)

0 :  ?
1 :  Cambodia
2 :  Canada
3 :  China
4 :  Columbia
5 :  Cuba
6 :  Dominican-Republic
7 :  Ecuador
8 :  El-Salvador
9 :  England
10 :  France
11 :  Germany
12 :  Greece
13 :  Guatemala
14 :  Haiti
15 :  Holand-Netherlands
16 :  Honduras
17 :  Hong
18 :  Hungary
19 :  India
20 :  Iran
21 :  Ireland
22 :  Italy
23 :  Jamaica
24 :  Japan
25 :  Laos
26 :  Mexico
27 :  Nicaragua
28 :  Outlying-US(Guam-USVI-etc)
29 :  Peru
30 :  Philippines
31 :  Poland
32 :  Portugal
33 :  Puerto-Rico
34 :  Scotland
35 :  South
36 :  Taiwan
37 :  Thailand
38 :  Trinadad&Tobago
39 :  United-States
40 :  Vietnam
41 :  Yugoslavia


In [7]:
df_orig.iloc[:,[3,8,9,13,14]]

,education,race,sex,native-country\t,class
0,Bachelors,White,Male,39,0
1,Bachelors,White,Male,39,0
2,HS-grad,White,Male,39,0
3,11th,Black,Male,39,0
4,Bachelors,Black,Female,5,0
...,...,...,...,...,...
32556,Assoc-acdm,White,Female,39,0
32557,HS-grad,White,Male,39,1
32558,HS-grad,White,Female,39,0
32559,HS-grad,White,Male,39,0


# 1. Project Proposal


We pulled the data from https://archive-beta.ics.uci.edu/dataset/2/adult .

## Correlation Matrix

In [ ]:
df_small = df_orig.iloc[:,[8,9,13,14]]
correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()

# 2. Data Collection and data cleaning


# 3. EDA

# 4. Techniques Used

# 5. Project Proposal